exam the similarity metric difference
1. representation cosine
2. EL2N
3. Grad Norm

In [1]:
import os, sys
import json
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

from scipy.spatial import distance
from scipy import stats
from scipy.special import softmax

sys.path.append("..")
from singleVis.data import NormalDataProvider
from singleVis.utils import find_neighbor_preserving_rate

In [2]:
VIS_METHOD = "tDVI" # DeepVisualInsight
CONTENT_PATH = "/home/xianglin/projects/DVI_data/resnet18_mnist"
GPU_ID = "0"

In [3]:
sys.path.append(CONTENT_PATH)
with open(os.path.join(CONTENT_PATH, "config.json"), "r") as f:
    config = json.load(f)
config = config[VIS_METHOD]

In [4]:
SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
PREPROCESS = config["VISUALIZATION"]["PREPROCESS"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
VIS_MODEL = VISUALIZATION_PARAMETER['VIS_MODEL']
LAMBDA1 = VISUALIZATION_PARAMETER["LAMBDA1"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]

VIS_MODEL_NAME = VISUALIZATION_PARAMETER["VIS_MODEL_NAME"]
EVALUATION_NAME = VISUALIZATION_PARAMETER["EVALUATION_NAME"]

# Define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

In [5]:
VIS_MODEL_NAME = "tDVI_baseAE"
EVALUATION_NAME = "evaluation_tDVI_baseAE"

In [6]:
import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))

In [7]:
# Define data_provider
data_provider = NormalDataProvider(CONTENT_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, device=DEVICE, classes=CLASSES, epoch_name="Epoch", verbose=1)
if PREPROCESS:
    data_provider._meta_data()
    if B_N_EPOCHS >0:
        data_provider._estimate_boundary(LEN//10, l_bound=L_BOUND)

Finish initialization...


# Define semantic change

In [8]:
def gradient_diff(prev_e, next_e, x, target, data_provider, criterion):

    model_t = data_provider.model_function(prev_e)
    model_t = model_t.to(DEVICE)
    optimizer = optim.SGD(model_t.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

    # Forward pass and compute gradients at time t
    output_t = model_t(x)
    loss_t = criterion(output_t, target)
    optimizer.zero_grad()
    loss_t.backward()

    # Save gradients at time t
    grads_t = [p.grad.clone() for p in model_t.parameters()]


    model_t1 = data_provider.model_function(next_e)
    model_t1 = model_t1.to(DEVICE)
    optimizer = optim.SGD(model_t1.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    # Forward pass and compute gradients at time t+1
    output_t1 = model_t1(x)
    loss_t1 = criterion(output_t1, target)
    optimizer.zero_grad()
    loss_t1.backward()

    # Save gradients at time t+1
    grads_t1 = [p.grad.clone() for p in model_t1.parameters()]

    # Compute cosine similarity between gradients at t and t+1
    cos_sim_values = []
    cos = nn.CosineSimilarity(dim=0)
    for g_t, g_t1 in zip(grads_t, grads_t1):
        cos_sim = cos(g_t.flatten(), g_t1.flatten())
        cos_sim_values.append(cos_sim.item())

    # Average cosine similarity
    avg_cos_sim = sum(cos_sim_values) / len(cos_sim_values)

    # Compute cosine distance
    cos_dist = 1 - avg_cos_sim

    # print(f"Cosine Distance: {cos_dist}")
    return cos_dist

In [25]:
prev_e = 13
next_e = 15

In [26]:
criterion = nn.CrossEntropyLoss()

training_data = data_provider._training_data()
targets = data_provider.train_labels(prev_e)

test_len = 100
idxs = np.random.choice(len(training_data), test_len, replace=False)

dists = np.zeros(test_len)
for i in range(test_len):
    x = training_data[idxs[i]:idxs[i]+1]
    y = torch.from_numpy(targets[idxs[i]:idxs[i]+1]).to(DEVICE)
    dist = gradient_diff(prev_e, next_e, x, y, data_provider, criterion)
    dists[i] = dist

In [27]:
# compute EL2N
prev_data = data_provider.train_representation(prev_e)
next_data = data_provider.train_representation(next_e)
train_labels = data_provider.train_labels(prev_e)
prev_pw = data_provider.get_pred(next_e, prev_data)
next_pw = data_provider.get_pred(next_e, next_data)
y = np.eye(np.max(train_labels)+1)[train_labels]

prev_pw = softmax(prev_pw, axis=1)
next_pw = softmax(next_pw, axis=1)
prev_el2n = prev_pw-y
next_el2n = next_pw-y

100%|██████████| 300/300 [00:00<00:00, 6559.44it/s]


In [28]:
print("Temporal repr")
repr_dists = np.array([distance.cosine(prev_data[idxs[i]], next_data[idxs[i]]) for i in range(len(idxs))])
repr_dists_eu = np.array([distance.euclidean(prev_data[idxs[i]], next_data[idxs[i]]) for i in range(len(idxs))])
stats.pearsonr(repr_dists, dists), stats.pearsonr(repr_dists_eu, dists)

Temporal repr


(PearsonRResult(statistic=-0.23657691998611244, pvalue=0.017799136928073844),
 PearsonRResult(statistic=-0.1330303782189574, pvalue=0.18701824340591475))

In [29]:
# z-score similarity calculation
from scipy import stats
prev_norm = stats.zscore(prev_data)
next_norm = stats.zscore(next_data)
norm_repr_dists_eu = np.array([distance.euclidean(prev_norm[idxs[i]], next_norm[idxs[i]]) for i in range(len(idxs))])
stats.pearsonr(norm_repr_dists_eu, dists)

PearsonRResult(statistic=-0.13060886087056184, pvalue=0.1952430285675027)

In [ ]:
# # neighbor preserving rate
# npr_eu = find_neighbor_preserving_rate(prev_data[idxs], next_data[idxs], n_neighbors=15, metric="euclidean")
# npr_cosine = find_neighbor_preserving_rate(prev_data[idxs], next_data[idxs], n_neighbors=15, metric="cosine")
# stats.spearmanr(npr_eu, dists), stats.spearmanr(npr_cosine, dists)

# evaluate

In [ ]:
criterion = nn.CrossEntropyLoss()

training_data = data_provider._training_data()
targets = data_provider.train_labels(1)

test_len = 10
idxs = np.random.choice(len(training_data), test_len, replace=False)

In [ ]:
data = training_data[idxs]
labels = torch.from_numpy(targets[idxs]).to(DEVICE)

start = data_provider.s
end = data_provider.e
period = data_provider.p
LEN = len(idxs)
EPOCH = (end - start) // period + 1

In [ ]:
high_diff = np.zeros((test_len, EPOCH-1))
for i in range(test_len):
    for prev_e, next_e in zip(range(start, EPOCH), range(start+period,EPOCH+1)):
        cos_diff = gradient_diff(prev_e, next_e, data[i:i+1], labels[i:i+1], data_provider, criterion)
        high_diff[i, prev_e-1] = cos_diff
high_diff

array([[0.41953571, 0.27321391, 0.24926051, 0.27854855, 0.36799557,
        0.42341644, 0.38818385, 0.36316491, 0.58657772, 0.50273828,
        0.25421803, 0.17575124, 0.09855688, 0.06827186],
       [0.39179168, 0.26227702, 0.2909557 , 0.29576715, 0.35235029,
        0.43536029, 0.3941605 , 0.34470697, 0.32502916, 0.31006617,
        0.24018108, 0.16255178, 0.08585598, 0.07388178],
       [0.32083815, 0.26364554, 0.23442972, 0.26694593, 0.30454997,
        0.34250334, 0.41704744, 0.40726463, 0.3643499 , 0.28702517,
        0.17327915, 0.16932114, 0.0846475 , 0.04831351],
       [0.30103821, 0.27762048, 0.27158107, 0.3089407 , 0.27938881,
        0.50409725, 0.4930542 , 0.30367474, 0.49879786, 0.40805532,
        0.23286593, 0.18255562, 0.11210328, 0.0729455 ],
       [0.42239682, 0.26496278, 0.24439643, 0.23824394, 0.29390182,
        0.36091006, 0.36284533, 0.35847531, 0.38422423, 0.41403395,
        0.23063892, 0.14810225, 0.09806933, 0.06349607],
       [0.37134237, 0.31097241, 0.2

In [ ]:
from singleVis.projector import DVIProjector
from singleVis.vis_models import vis_models as vmodels
VIS_MODEL_NAME = "tDVI_baseAE"
EVALUATION_NAME = "evaluation_singleDVI_baseAE"
VIS_MODEL = "baseAE"
model = vmodels[VIS_MODEL](ENCODER_DIMS, DECODER_DIMS)
projector = DVIProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name=VIS_MODEL_NAME, epoch_name="Epoch", device=DEVICE)

2023-08-29 16:17:46.255031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 16:17:46.730767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
from singleVis.projector import tfDVIProjector
flag = "_temporal_id_withoutB"
projector = tfDVIProjector(CONTENT_PATH, flag=flag)

In [ ]:
from singleVis.projector import TimeVisProjector
from singleVis.SingleVisualizationModel import VisModel
ENCODER_DIMS = [512,256,256,256,256,256,2]
DECODER_DIMS = [2,256,256,256,256,256,512]
ENCODER_DIMS = [512,256,2]
DECODER_DIMS = [2,256,512]
VIS_MODEL_NAME = "timevis"
model = VisModel(ENCODER_DIMS, DECODER_DIMS)
projector = TimeVisProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name=VIS_MODEL_NAME, device=DEVICE)

In [ ]:
low_repr = np.zeros((EPOCH,LEN,2))
for i in range(start,end + 1, period):
    index = (i - start) //  period
    low_repr[index] = projector.batch_project(i, data_provider.train_representation(i)[idxs]);

/home/xianglin/projects/DVI_data/resnet18_mnist/Model/Epoch_1/tDVI_baseAE.pth
Successfully load the DVI visualization model for iteration 1
/home/xianglin/projects/DVI_data/resnet18_mnist/Model/Epoch_2/tDVI_baseAE.pth
Successfully load the DVI visualization model for iteration 2
/home/xianglin/projects/DVI_data/resnet18_mnist/Model/Epoch_3/tDVI_baseAE.pth
Successfully load the DVI visualization model for iteration 3
/home/xianglin/projects/DVI_data/resnet18_mnist/Model/Epoch_4/tDVI_baseAE.pth
Successfully load the DVI visualization model for iteration 4
/home/xianglin/projects/DVI_data/resnet18_mnist/Model/Epoch_5/tDVI_baseAE.pth
Successfully load the DVI visualization model for iteration 5
/home/xianglin/projects/DVI_data/resnet18_mnist/Model/Epoch_6/tDVI_baseAE.pth
Successfully load the DVI visualization model for iteration 6
/home/xianglin/projects/DVI_data/resnet18_mnist/Model/Epoch_7/tDVI_baseAE.pth
Successfully load the DVI visualization model for iteration 7
/home/xianglin/proje

In [ ]:
low_repr = low_repr.transpose([1,0,2])
low_dists = np.linalg.norm(low_repr[:,start//period:,:]-low_repr[:,:(end-period)//period,:], axis=2)
low_dists.shape

(10, 14)

In [ ]:
corrs = np.zeros(LEN)
ps = np.zeros(LEN)
for i in range(LEN):
    corr, p = stats.spearmanr(high_diff[i], low_dists[i])
    corrs[i] = corr
    ps[i] = p
print(f"Global temporal ranking #train:{corrs.mean()}")

Global temporal ranking #train:0.3758241758241759


In [ ]:
corrs = np.zeros(LEN)
ps = np.zeros(LEN)
for i in range(LEN):
    corr, p = stats.spearmanr(high_diff[i], low_dists[i])
    corrs[i] = corr
    ps[i] = p
print(f"DVI: Global temporal ranking #train:{corrs.mean()}")

DVI: Global temporal ranking #train:0.3147252747252748


In [ ]:
corrs = np.zeros(LEN)
ps = np.zeros(LEN)
for i in range(LEN):
    corr, p = stats.spearmanr(high_diff[i], low_dists[i])
    corrs[i] = corr
    ps[i] = p
print(f"TimeVis: Global temporal ranking #train:{corrs.mean()}")

TimeVis: Global temporal ranking #train:0.531868131868132
